In [103]:
import pandas as pd

reviews = pd.read_json('data/ys-reviews-with-categories.json')

restuarants = reviews.loc[reviews.category == "restaurant"]

sample: list[str] = restuarants.sample(10000).text.tolist()

In [107]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

import yake as yk

stop_words = stopwords.words("english")
keyword_extractor = yk.KeywordExtractor(n=3,top=1000)
filtered_words = []

# def key_ngram_per_sentence (review_text) -> list[str]:
#     sentences = sent_tokenize(review_text)
#     key_ngrams = []
#     for sentence in sentences:
#         keywords = keyword_extractor.extract_keywords(removed_stopwords(sentence))
#         if len(keywords) > 0: key_ngrams.append(keywords[0][0])
#     return key_ngrams    
    
# def key_ngram (review_text) -> list[str]:
#     key_ngrams = []
#     keywords = keyword_extractor.extract_keywords(removed_stopwords(review_text))
#     if len(keywords) > 0: key_ngrams.append(keywords[0][0])
#     return key_ngrams    
    
def removed_stopwords(text: str) -> str:
    # make lower case and remove all punctuations
    text = text.lower().translate(str.maketrans('', '', punctuation))
    tokens = word_tokenize(text)
    # filter stopwords    
    filtered = [word for word in tokens if word not in stop_words]
    filtered = [word for word in tokens if word not in filtered_words]
    return ' '.join(filtered)


corpus = "\n".join(sample)
corpus = removed_stopwords(corpus)

extractions = keyword_extractor.extract_keywords(corpus)

with open('keywords.csv', 'w') as outfile:
    outfile.write(pd.DataFrame(extractions).to_csv(index=False))

# keywords = list(set([extraction[0] for extraction in extractions]))

# keywords

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Sample trigrams
trigrams = keywords

# Convert trigrams to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(trigrams)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Perform hierarchical clustering
n_clusters = 7  # You can adjust the number of clusters
clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='cosine', linkage='average')
cluster_labels = clustering.fit_predict(cosine_sim_matrix)

# Assign trigrams to clusters
trigram_clusters = {}
for trigram, label in zip(trigrams, cluster_labels):
    if label not in trigram_clusters:
        trigram_clusters[label] = []
    trigram_clusters[label].append(trigram)

# Print the trigram clusters
for label, cluster in trigram_clusters.items():
    if len(cluster) > 1 : print(f'Cluster {label + 1}: {cluster}')


Cluster 1: ['good food awesome', 'food good', 'good food fast', 'great fresh food', 'good tacos good', 'good food easy', 'good food nice', 'good service nice', 'great food fast', 'good food pizza', 'good food drink', 'flavor good food', 'good food bar', 'yummy food good', 'fries good food', 'sandwich food nice', 'good great', 'food tasted good', 'good service fast', 'fresh good food', 'good food love', 'good food open', 'great pretty good', 'good stuff food', 'good restaurant good', 'fresh food good', 'pretty good great', 'service pretty good', 'pretty good', 'fast food places', 'extremely good food', 'service nice place', 'food fast service', 'great food good', 'restaurant good food', 'good food portions', 'good great selection', 'good food items', 'good fast food', 'bar good food', 'good food great', 'fast food good', 'clean good food', 'food good drinks', 'good beer good', 'good food good', 'good food', 'dinners good food', 'kind good food', 'food good food', 'great good california'